In [1]:
import torch
from torch import nn
import torchvision ## For all your CV task
import matplotlib.pyplot as plt

D:\Computer_Vision\envs\detectron2\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DATA_ROOT_DIR = "FashionMnist"

In [5]:
train_data = torchvision.datasets.FashionMNIST(
    root = DATA_ROOT_DIR,
    train = True,
    download = True,
    transform = torchvision.transforms.ToTensor(),
)

test_data = torchvision.datasets.FashionMNIST(
    root = DATA_ROOT_DIR,
    train = False,
    download = True,
    transform = torchvision.transforms.ToTensor(),
)

In [6]:
BATCH_SIZE = 64

train_data_loader = torch.utils.data.DataLoader(train_data, batch_size = BATCH_SIZE)  # Like keras flow from directory
test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE)

In [7]:
type(train_data_loader)

torch.utils.data.dataloader.DataLoader

In [8]:
for X, y in test_data_loader:
    print(X.shape)
    print(y.shape, y.dtype, torch.unique(y))
    break

torch.Size([64, 1, 28, 28])
torch.Size([64]) torch.int64 tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [30]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        
        self.flatten = nn.Flatten()
        
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),  # Just like dense layer
            nn.ReLU(), # Activation function
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU(),
        )
        
    
    def forward(self, X):  # Forward Propagation
        X = self.flatten(X)
        logits = self.linear_relu_stack(X)
        return logits

In [31]:
model = NeuralNetwork().to(device)

print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [32]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)  # model.parameter is weights and biases

In [33]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)  # Convert to GPU
        
        # Calculate Error
        pred = model(X)     # Predict
        loss = loss_fn(pred, y)   # Loss function
        
        # Backpropagation
        optimizer.zero_grad()  # No cache gradients
        loss.backward()  # Calculate Gradient
        optimizer.step() # Update the weights 
        
        if batch % 100 == 0:  # Every 6400th step
            loss, current = loss.item(), batch * len(X)  # Current Data Point
            print(f"Training Loss: {loss} [{current}/{size}]")

In [34]:
def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    
    test_loss, correct = 0, 0 
    
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss = test_loss + loss_fn(pred, y).item()
            correct = correct + (pred.argmax(1)==y).type(torch.float).sum().item()
    
    test_loss = test_loss/size
    correct = correct/size
    
    print(f"Test Loss: {100*correct}, Average Loss: {test_loss}")

In [35]:
EPOCHS = 3

for epoch in range(EPOCHS):
    print(f"EPOCH: {epoch + 1}\n------------")
    
    train(train_data_loader, model, loss_fn, optimizer)
    test(test_data_loader, model)
    
print("Done")

EPOCH: 1
------------
Training Loss: 2.3023295402526855 [0/60000]
Training Loss: 2.297351360321045 [6400/60000]
Training Loss: 2.288860559463501 [12800/60000]
Training Loss: 2.2813501358032227 [19200/60000]
Training Loss: 2.2841525077819824 [25600/60000]
Training Loss: 2.257342576980591 [32000/60000]
Training Loss: 2.272545576095581 [38400/60000]
Training Loss: 2.25211238861084 [44800/60000]
Training Loss: 2.227989912033081 [51200/60000]
Training Loss: 2.2291555404663086 [57600/60000]
Test Loss: 38.43, Average Loss: 0.034958814549446104
EPOCH: 2
------------
Training Loss: 2.2252542972564697 [0/60000]
Training Loss: 2.2234432697296143 [6400/60000]
Training Loss: 2.1947224140167236 [12800/60000]
Training Loss: 2.2056076526641846 [19200/60000]
Training Loss: 2.1895296573638916 [25600/60000]
Training Loss: 2.1248345375061035 [32000/60000]
Training Loss: 2.1706247329711914 [38400/60000]
Training Loss: 2.1127569675445557 [44800/60000]
Training Loss: 2.0923104286193848 [51200/60000]
Training

In [36]:
torch.save(model, "model.pth")

In [38]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()

X, y = torch.tensor(test_data[0][0]).to(device), torch.tensor(test_data[0][1]).to(device)

with torch.no_grad():
    pred = model(X)
    prediction, actual = classes[pred[0].argmax(0)], classes[y]
    print(prediction, actual)

D:\Computer_Vision\envs\detectron2\lib\site-packages\ipykernel_launcher.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


Ankle boot Ankle boot
